## Infinite number of monkeys thought experiment

The [infinite monkey theorem](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) states that a monkey hitting keys at random on a typewriter keyboard for an infinite amount of time will almost surely type any given text, such as the complete works of William Shakespeare.

We'll make some simplifications. We'll assume that the monkeys:
* can only type the letters a-z, A-Z, space, and a few punctuation
* we are only looking for them to complete "To be or not to be?"

### Using Jenetics

Let's consider a solution using the [Jenetics](https://jenetics.io/) library.

In [1]:
%%classpath add mvn
io.jenetics jenetics 5.2.0

With our `limit` of `10000`, the result may not always be fully complete.
You can increase that limit to ensure you always get the correct result.

In [2]:
%%groovy

import io.jenetics.*
import io.jenetics.engine.*
import java.util.function.*

def SENTENCE = "To be or not to be?"
int DIMENSION = SENTENCE.size()
def POSSIBLE_CHARS = ('a'..'z') + ('A'..'Z') + " !?.".toList()
int SIZE = POSSIBLE_CHARS.size()
def r = new Random()

Supplier generateRandomChar = { -> POSSIBLE_CHARS[r.nextInt(SIZE)] as char }

Function fitness = { Genotype gt ->
    (0..<DIMENSION).inject(0) { sum, i ->
        def target = SENTENCE.charAt(i)
        def allele = gt.chromosome().get(i).allele()
        int exactBonus = allele == target ? 100 : 0
        sum + SIZE - (target - allele).abs() + exactBonus
    }
}

def gtf = Genotype.of(AnyChromosome.of(generateRandomChar, DIMENSION), new Chromosome[]{})
def engine = Engine.builder(fitness, gtf).offspringSelector(new RouletteWheelSelector()).build()
def result = engine.stream().limit(10000).collect(EvolutionResult.toBestGenotype())
println "Result: $result" // Result: [[T,o, ,b,e, ,o,r, ,o,o,t, ,t,o, ,b,e,?]]

OutputCell.HIDDEN